Load Libraries and Data

In [2]:
import pandas as pd
import numpy as np
import os 
from pathlib import Path
from PIL import Image

TEST_IMAGE_DIR = "./dog-breed-identification/test/"
TRAIN_IMAGE_DIR = "./dog-breed-identification/train/"
LABELS = './dog-breed-identification/labels.csv'

data = pd.read_csv(LABELS)

print(data)

                                     id                     breed
0      000bec180eb18c7604dcecc8fe0dba07               boston_bull
1      001513dfcb2ffafc82cccf4d8bbaba97                     dingo
2      001cdf01b096e06d78e9e5112d419397                  pekinese
3      00214f311d5d2247d5dfe4fe24b2303d                  bluetick
4      0021f9ceb3235effd7fcde7f7538ed62          golden_retriever
...                                 ...                       ...
10217  ffd25009d635cfd16e793503ac5edef0                    borzoi
10218  ffd3f636f7f379c51ba3648a9ff8254f            dandie_dinmont
10219  ffe2ca6c940cddfee68fa3cc6c63213f                  airedale
10220  ffe5f6d8e2bff356e9482a80a6e29aac        miniature_pinscher
10221  fff43b07992508bc822f33d8ffd902ae  chesapeake_bay_retriever

[10222 rows x 2 columns]


In [3]:
#Data from the csv only contans the ids of the photos and not the photos itself
#Iterate through the data and check whether the photo is in test or train
#Then fetch it and store it in its proper variable
import tensorflow as tf
from os.path import join
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input


image_size = 224

def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img).flatten() for img in imgs])
    output =  preprocess_input(img_array)
    return(output)

imagepaths = [join(TRAIN_IMAGE_DIR, filename + ".jpg") for filename in data["id"]]

X = read_and_prep_images(imagepaths)


In [4]:
print(X.shape)
y = data["breed"].to_numpy()

(10222, 150528)


Create the Train and Test Variables

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
#Method that will show you the picture from the given matrix
def showImage(imageMatrix):
    image = Image.fromarray(imageMatrix)
    image.show()

Create The Decision Tree Model

In [7]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

clf  = DecisionTreeClassifier(random_state=42, max_depth=2)

'''from sklearn.svm import SVC

# SVM Classifier model
clf = SVC(kernel="linear")
clf.fit(X_train, y_train)'''


'from sklearn.svm import SVC\n\n# SVM Classifier model\nclf = SVC(kernel="linear")\nclf.fit(X_train, y_train)'

In [8]:
print(clf)

DecisionTreeClassifier(max_depth=2, random_state=42)


In [9]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=2, random_state=42)

In [10]:
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.01613691931540342
